In [1]:
## Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Custom Functions for Analysis

In [2]:
def col_summary(df, col, dropna=False):
    
    """Takes in a Pandas DataFrame and specific column name. 
    Prints the number of unique values in the column and displays
    a DataFrame with the 5 most common and 5 least common values
    in that column as well as the count of each value. 
    Default is to also provide a count of NaN values.
    
    Args:
        df (DataFrame): DataFrame containing the column to summarize.
        col (str): Name of the column to be summarized.
        dropna (bool, default=False): Whether or not to drop null values.
    
    Example:
        >>> df = pd.DataFrame({'a': [2, 4, 4, 6],
                               'b': [2, 1, 3, 4]})
        >>> col_summary(df, col='a', dropna=False)
        
        ******************************
        Summary of a
        ******************************
        Total unique values: 3
        
            count 
        4   2  
        6   1
        2   1 
    """
    
    pd.options.display.max_rows = 10
    
    print('***'*10)
    print(f"Summary of {col}")
    print('***'*10)
    print(f"Total unique values: {df[col].nunique()}")
    
    unique_vals = pd.DataFrame()
    unique_vals['count'] = pd.Series(df[col].value_counts(dropna=dropna))
    display(unique_vals)

# Obtain

In [3]:
## Read in data files
popn_df = pd.read_csv('share-of-population-urban.csv')
gdp_df = pd.read_csv('taxes-on-incomes-of-individuals-and-corporations-gdp.csv')

In [4]:
## Inspect first 5 rows of each df
display(popn_df.head())
gdp_df.head()

,Entity,Code,Year,Urban_Population
0,Afghanistan,AFG,1960,8.401
1,Afghanistan,AFG,1961,8.684
2,Afghanistan,AFG,1962,8.976
3,Afghanistan,AFG,1963,9.276
4,Afghanistan,AFG,1964,9.586


,Entity,Code,Year,Tax_Percent_GDP
0,Afghanistan,AFG,2003,0.165953
1,Afghanistan,AFG,2004,0.411647
2,Afghanistan,AFG,2005,0.320864
3,Afghanistan,AFG,2006,1.261181
4,Afghanistan,AFG,2007,1.323461


## Initial Exploration of Population Dataset

The urban population dataset only has null values in the `Code` column. This likely shouldn't be an issue and I'll look to join the two DataFrames on `Entity` and then `Year` after further investigation.

<br> 
The data types for each column make sense for the information they contain, except in the case of `Urban_Population`. This column reports the percentage of the total population that lives in urban areas, and thus should be a float. However, it is an object dtype. This will need to be converted to a numeric data type to enable regression analysis of the effect of `Urban_Population` on `Tax_Percent_GDP`. 
- Attempting to recast this column as type 'float64' produced the following error: 
```ValueError: could not convert string to float: '88%'```
- Removing '%'s from values in this column solved the issue.

<br> 
The `Entity` column of the population dataset includes some completely nonsensical values (e.g., "43hj43"), as well as values that are not countries (e.g., "OECD members" and "Upper middle income"). These values may not appear in the GDP dataset and would thus be dropped through an inner join on `Entity`. This will require further investigation once the files have been merged.

In [5]:
## Metadata for urban population dataset
popn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15072 entries, 0 to 15071
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Entity            15072 non-null  object
 1   Code              12404 non-null  object
 2   Year              15072 non-null  int64 
 3   Urban_Population  15072 non-null  object
dtypes: int64(1), object(3)
memory usage: 471.1+ KB


In [6]:
## Recast Urban_Population as a float
popn_df['Urban_Population'] = popn_df['Urban_Population'].str.replace('%','').astype('float64')

In [7]:
## Summary statistics of the numeric columns of popn_df
popn_df.describe()

,Year,Urban_Population
count,15072.000000,15072.000000
mean,1988.545382,49.525735
std,16.737532,24.818031
min,1960.000000,2.077000
25%,1974.000000,29.180000
50%,1989.000000,47.782500
75%,2003.000000,68.870750
max,2017.000000,100.000000


In [8]:
## Investigate type and prevalence of unique values in each column of popn_df
for col in list(popn_df.columns):
    col_summary(popn_df, col)
    print('---'*5)
    print('\n\n')

******************************
Summary of Entity
******************************
Total unique values: 270


,count
Mexico,58
Belize,58
Venezuela,58
Bhutan,58
Vanuatu,58
...,...
rehojher,1
sjke,1
rreji43,1
herger,1


---------------



******************************
Summary of Code
******************************
Total unique values: 215


,count
NaN,2668
TGO,58
YEM,58
GEO,58
VCT,58
...,...
LVA,58
NRU,58
ERI,52
SRB,28


---------------



******************************
Summary of Year
******************************
Total unique values: 58


,count
2003,261
2011,261
2004,261
2007,261
1995,261
...,...
1987,259
1965,259
1988,259
1964,259


---------------



******************************
Summary of Urban_Population
******************************
Total unique values: 13664


,count
100.000,468
21.200,10
83.100,10
79.800,8
90.400,7
...,...
60.983,1
23.081,1
68.911,1
37.399,1


---------------





> Upon investigation of the following countries/regions, it does appear that roughly 100% of the total population does in fact live in urban areas. This is not a filler value.

In [9]:
## Check which countries report 100% of population lives in urban areas
col_summary(popn_df.loc[popn_df['Urban_Population']==100], 'Entity')

******************************
Summary of Entity
******************************
Total unique values: 10


,count
Sint Maarten (Dutch part),58
Bermuda,58
Cayman Islands,58
Gibraltar,58
Monaco,58
Singapore,58
Nauru,58
Hong Kong,25
Macao,21
Kuwait,16


## Initial Exploration of GDP Dataset

The GDP dataset has no null values and the data type for each column aligns with the type of information in the column.

Upon initial inspection, the `Entity` column in this GDP dataset appears cleaner than that in the population dataset.

In [10]:
## Metadata for GDP dataset
gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4871 entries, 0 to 4870
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Entity           4871 non-null   object 
 1   Code             4871 non-null   object 
 2   Year             4871 non-null   int64  
 3   Tax_Percent_GDP  4871 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 152.3+ KB


In [11]:
## Summary statistics of the numeric columns of gdp_df
gdp_df.describe()

,Year,Tax_Percent_GDP
count,4871.000000,4871.000000
mean,2000.872716,6.538407
std,9.803376,4.915040
min,1980.000000,0.000000
25%,1994.000000,2.903660
50%,2001.000000,5.455395
75%,2009.000000,8.992646
max,2017.000000,32.075305


In [12]:
## Investigate type and prevalence of unique values in each column of gdp_df
for col in list(gdp_df.columns):
    col_summary(gdp_df, col)
    print('---'*5)
    print('\n\n')

******************************
Summary of Entity
******************************
Total unique values: 186


,count
Thailand,38
Dominican Republic,38
Canada,38
Denmark,38
Costa Rica,38
...,...
Somalia,5
Kosovo,4
Iran,2
Algeria,2


---------------



******************************
Summary of Code
******************************
Total unique values: 186


,count
CYP,38
MAR,38
ESP,38
BEL,38
IRL,38
...,...
SOM,5
OWID_KOS,4
BHS,2
DZA,2


---------------



******************************
Summary of Year
******************************
Total unique values: 38


,count
2002,169
2003,167
2001,166
2000,166
2004,166
...,...
1984,73
1983,71
1982,64
1980,61


---------------



******************************
Summary of Tax_Percent_GDP
******************************
Total unique values: 4853


,count
0.000000,16
5.176471,2
8.035714,2
2.760678,2
5.150947,1
...,...
12.528528,1
15.845956,1
12.751597,1
4.302662,1


---------------





## Merge the DataFrames

In [13]:
## Create new DataFrame object by merging the 2 datasets
social_df = pd.merge(
    left=gdp_df, right=popn_df,
    
    # Perform inner join to retain only the rows present in both files
    how='inner',
    
    # Joining first on country, then on year
    on=['Entity', 'Year'],
   
    # Check that there are not duplicate keys (combos of Entity and Year)
    validate='one_to_one'
)

social_df.head()

,Entity,Code_x,Year,Tax_Percent_GDP,Code_y,Urban_Population
0,Afghanistan,AFG,2003,0.165953,AFG,22.353
1,Afghanistan,AFG,2004,0.411647,AFG,22.500
2,Afghanistan,AFG,2006,1.261181,AFG,22.907
3,Afghanistan,AFG,2007,1.323461,AFG,23.113
4,Afghanistan,AFG,2008,1.664719,AFG,23.320


In [14]:
## Metadata for social dataset
social_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4822 entries, 0 to 4821
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Entity            4822 non-null   object 
 1   Code_x            4822 non-null   object 
 2   Year              4822 non-null   int64  
 3   Tax_Percent_GDP   4822 non-null   float64
 4   Code_y            4822 non-null   object 
 5   Urban_Population  4822 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 263.7+ KB


In [15]:
## Drop one of the Code columns and rename remaining
social_df.drop(columns=['Code_y'], inplace=True)
social_df.rename(columns={'Code_x': 'Code'}, inplace=True)
social_df.head()

,Entity,Code,Year,Tax_Percent_GDP,Urban_Population
0,Afghanistan,AFG,2003,0.165953,22.353
1,Afghanistan,AFG,2004,0.411647,22.500
2,Afghanistan,AFG,2006,1.261181,22.907
3,Afghanistan,AFG,2007,1.323461,23.113
4,Afghanistan,AFG,2008,1.664719,23.320


In [16]:
## Summary statistics of the numeric columns of social_df
social_df.describe()

,Year,Tax_Percent_GDP,Urban_Population
count,4822.000000,4822.000000,4822.000000
mean,2000.815844,6.541968,54.562002
std,9.825130,4.914286,24.089673
min,1980.000000,0.000000,4.674000
25%,1994.000000,2.913589,34.030750
50%,2001.000000,5.453532,54.978000
75%,2009.000000,8.964636,74.338750
max,2017.000000,32.075305,100.000000


In [17]:
## Investigate type and prevalence of unique values in each column of social_df
for col in list(social_df.columns):
    col_summary(social_df, col)
    print('---'*5)
    print('\n\n')

******************************
Summary of Entity
******************************
Total unique values: 182


,count
Thailand,38
Belgium,38
Nepal,38
Venezuela,38
Mauritius,38
...,...
Congo,6
Somalia,5
Iran,2
Algeria,2


---------------



******************************
Summary of Code
******************************
Total unique values: 182


,count
THA,38
ESP,38
CYP,38
FIN,38
VEN,38
...,...
COG,6
SOM,5
IRN,2
BHS,2


---------------



******************************
Summary of Year
******************************
Total unique values: 38


...

---------------



******************************
Summary of Tax_Percent_GDP
******************************
Total unique values: 4815


,count
0.000000,5
8.035714,2
5.176471,2
2.760678,2
11.306637,1
...,...
5.474110,1
7.046936,1
3.071036,1
12.528528,1


---------------



******************************
Summary of Urban_Population
******************************
Total unique values: 4576


,count
100.000,89
83.100,10
22.330,6
74.644,3
83.143,3
...,...
74.286,1
66.726,1
70.241,1
49.683,1


---------------





This new merged DataFrame now appears to be in good shape and ready for export to csv.

In [18]:
## Export merged df to csv
social_df.to_csv('merged_social_data.csv', index=False)

In [19]:
social_df = pd.read_csv('merged_social_data.csv')
social_df.head()

,Entity,Code,Year,Tax_Percent_GDP,Urban_Population
0,Afghanistan,AFG,2003,0.165953,22.353
1,Afghanistan,AFG,2004,0.411647,22.500
2,Afghanistan,AFG,2006,1.261181,22.907
3,Afghanistan,AFG,2007,1.323461,23.113
4,Afghanistan,AFG,2008,1.664719,23.320
